In [ ]:
import numpy as np
import pandas as pd
import json
import requests
import re
import binascii
import time
import sys

# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab Notebooks/
%ls


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks
Phrases1_2.csv     Phrases1.csv     phrases.gsheet
Phrases1_2.gsheet  Phrases1.gsheet  Untitled0.ipynb
Phrases1_3.csv     phrases.csv      Untitled1.ipynb


In [ ]:
s1= "bag"

n=8 #number of bit we will take per iteration
c=1 # constant to make random range bigger(must be greater than 1)
size_of_row=3 # average row size of each array
std_div =2
start = 0 
end=(2**n)*size_of_row*c #range of random numbers
float_dic = np.array([])
dic= float_dic.astype(int)# to store the generated unique random numbers that will be used to map words

map_int = [] # works as the 2-d array where each column will hold the random values for indexing
map_str_num= []
map_num_str= []
map_str_num_tmp=[]
encr_output=[]
list_num=[]
phrases=[]

In [ ]:
def new_rand(): # to generate random numbers
    global dic
    x=np.random.randint(start,end)
    res= np.where(dic==x)
    res= np.array(res)
    if res.size ==0:
      dic = np.append(dic,int(x)) #appending random number to dic if it is not already in dic
      return x; 
    return new_rand()


In [ ]:
def fillup_map_int(): 
    for i in range(0,2**n):
      random_len = size_of_row+ np.random.randint(-std_div,std_div) # taking random length for each column
      arr = []
      while random_len > 0:
        arr.append(new_rand())
        random_len=random_len-1
      map_int.append(arr)
  

In [ ]:
def copy_prob(phrase_arr):
  dupli= [0,0,0,0,0,0,0,0,0,0]
  for i in range(phrase_arr.shape[0]):
    cnt=0
    zx=str(phrase_arr[i])
    wrds = zx.split(' ')
    cur_wrd=''
    id=[]
    id_cnt=0
    for wrrd in wrds:
      cur_wrd = cur_wrd+wrrd
      if cur_wrd in map_str_num_tmp.keys():
        cnt=cnt+1
        id.append(id_cnt)
      if wrrd!= wrds[len(wrds)-1]:
        cur_wrd=cur_wrd+' '
      if cur_wrd in map_str_num_tmp.keys():
        cnt=cnt+1
        id.append(id_cnt)
      id_cnt=id_cnt+1
    dupli[cnt]=  dupli[cnt]+1
    if cnt>2:
      new_wrd=''
      for wrrd in wrds:
        new_wrd = new_wrd+wrrd
        if new_wrd in map_str_num_tmp.keys():
          new_wrd=new_wrd+'s'
        if wrrd!= wrds[len(wrds)-1]:
          new_wrd=new_wrd+' '
      phrase_arr[i]=new_wrd
  return phrase_arr

In [ ]:
def initialize():
  fillup_map_int()
  global map_str_num,map_num_str,phrases,map_str_num_tmp
  phrases=pd.read_csv("Phrases.csv",encoding= 'unicode_escape')
  phrases=phrases[0:len(dic)]
  prnp=np.array(phrases)
  prpn=prnp.reshape((len(dic)))
  #print(prpn)
  map_str_num_tmp = dict(zip(prpn, dic))
  prpn2=copy_prob(prpn)
  map_str_num = dict(zip(prpn2, dic))
  map_num_str = dict(zip(dic,prpn2))



In [ ]:
len(dic)

0

In [ ]:
initialize()
map_str_num

{'25 years later': 477,
 '8 percent of 25 is the': 684,
 'A good example': 649,
 'A kangaroo is an': 510,
 'A kangaroo is really': 381,
 'A penny saved': 309,
 'A quiet house': 647,
 'A suit of armor': 683,
 'Abstraction is often': 426,
 'Abstraction is oftens She tilted her head': 112,
 'After fighting off the': 516,
 'All she wanted was': 598,
 'All they could': 417,
 "Although it wasn't": 135,
 'Aluminum helps to': 576,
 "Art doesn't have": 663,
 'As he entered the church': 369,
 'As she walked': 404,
 'As she walkeds along': 342,
 'As you consider all': 377,
 'As you consider alls the': 653,
 'At last': 119,
 'At that moment': 321,
 'Australian kangaroo': 747,
 'Be careful with': 531,
 'Before he moved': 178,
 'Being unacquainted with': 51,
 'Best friends are': 243,
 'Bill ran from': 274,
 'Blue sounded': 700,
 'Blue soundeds too': 442,
 'Boulders lined the': 518,
 'Car safety systems': 0,
 'Carol drank the': 607,
 'Cats are good': 356,
 'Cats are goods pets': 537,
 'Check back tom

In [ ]:
def StrToBit(str):
  #from string to bit stream 
   return''.join(format(ord(i), '08b') for i in str)

In [ ]:
def bitsToNum(bits):#taking n bits at a time and converting it to decimal
  decim = []
  for i in range(0,len(bits),n):
    bit =bits[i:i+n]
    print(bit)
    dec=int(bit,2)
    print(dec)
    sz=len(map_int[dec])
    print(sz)
    id= np.random.randint(0,sz)
    decim.append(map_int[dec][id])
  return decim

In [ ]:
from pandas._libs import tslib
def generated_text(word,pos):
  
  r = requests.post(
      "https://api.deepai.org/api/text-generator",
      data={
          'text': word,
      },
      headers={'api-key': '9f35e24a-b4cc-4bcf-837d-ac10866fb122'}
  )
  s=r.json()
  s=s["output"]
  return s.split('.')[pos]

In [ ]:
def final_generated_output(list_num):
  stegano=[]

  last_sentence=''

  for num in list_num:
    word = map_num_str[num]
    new_sent=word
    current_sentence=''
    if last_sentence =='':
      current_sentence=generated_text(new_sent,0)+'.'
      
      
    else:
      new_sent=last_sentence+word
      current_sentence=generated_text(new_sent,1)+'.'
      
    id=[]
    for i in range(len(word),len(current_sentence)):
        if(current_sentence[i]==' ' or current_sentence[i]=='.'):
          break
        id.append(i)
    if len(id) >0:
     #print(current_sentence)
     current_sentence= word+current_sentence[id[len(id)-1]+1:len(current_sentence)]
     #print('paise')
     
    #print(word)
    stegano.append(current_sentence)
    last_sentence=current_sentence

  return stegano

In [ ]:
def encryption():
  global list_num, encr_output,s1
  bits=StrToBit(s1);
  print(bits)
  list_num=bitsToNum(bits);
  print(list_num)
  
  encr_output=final_generated_output(list_num)
  print(encr_output)

In [ ]:
encryption()

011000100110000101100111
01100010
98
4
01100001
97
3
01100111
103
2
[265, 666, 256]
['The ice cream trucks are in full stride.', "It's not often that I get the chance to experience a good local product or family time.", 'Traveling became so much more affordable and affordable, people just wanted to go all year long.']


In [ ]:
tstart=time.time()
encr_output=final_generated_output(list_num)
tend=time.time()
t=tend-tstart
  
print(encr_output)
print(t)


["The ice cream trucks are more like the people we're eating at work and the people who work for food trucks to me and the people you're spending money on — we're the people that I can spend money on.", "It's not often we get a real good answer to like the question, is there a good answer to this question when you are talking to other people because you ask, can we do this for us instead?When you ask, can I just make it to our restaurant and take the ice cream to the shop instead of the rest of the city to get to the restaurant instead for our meals? It turns out not.", 'Traveling became more expensive and it seems that sometimes all these problems can eventually lead to better restaurants that really work for you.']
6.705978870391846


In [ ]:
res=""
Li=0
Bsi=0
sum=0
for sntc in encr_output:
  res=res+sntc
  Li=len(sntc)-1
  print(Li)
  Bsi=sys.getsizeof(sntc)*8
  print(Bsi)
  sum=sum+(Li/Bsi)
  print(sum)
len(res)
print(sum/len(encr_output))

197
3760
0.052393617021276595
369
3352
0.162477149240847
145
1560
0.25542586718956495
0.08514195572985499


In [ ]:
map_int
res

"The ice cream trucks are more like the people we're eating at work and the people who work for food trucks to me and the people you're spending money on — we're the people that I can spend money on.It's not often we get a real good answer to like the question, is there a good answer to this question when you are talking to other people because you ask, can we do this for us instead?When you ask, can I just make it to our restaurant and take the ice cream to the shop instead of the rest of the city to get to the restaurant instead for our meals? It turns out not.Traveling became more expensive and it seems that sometimes all these problems can eventually lead to better restaurants that really work for you."

In [ ]:
def SentenceToNumber(res):
  whole = res.split('.')
  whole
  res_num=[]
  for combined_sent in whole:
    first_s = re.findall(r"[\w']+", combined_sent)
    #print(first_s)
    tmp_sent=''
    cnt=0;
    for single_word in first_s:
      if single_word =='':
        continue
      if cnt>0:
        tmp_sent=tmp_sent+' '
      tmp_sent=tmp_sent+single_word
      cnt=cnt+1
      if cnt>=2 and cnt<=7:
        print(tmp_sent)
        if tmp_sent in map_str_num:
          res_num.append(map_str_num[tmp_sent])
          break
      elif cnt>7:
        break
  print(res_num)
  return res_num

        
#problem
  



In [ ]:
a=res.split('.')
for i in range(len(a)):
  print(len(a[i].split()))

40
76
23
0


In [ ]:

if 'The secret code' in map_str_num:
  print(map_str_num['The secret code'])
else:
    print("Doesn't Work")

465


In [ ]:
def IndexToBinary():
  res_num=SentenceToNumber(res)
  index_res=''
  for j in res_num:
    for i in map_int:
      if(j in i):
        index_res=index_res+str(bin(map_int.index(i))[2:].zfill(n))
        print(map_int.index(i))
  return index_res

In [ ]:
def BinaryToChar():
  index_res=IndexToBinary();
  final_output=''
  for i in range(0,len(index_res),8):
      bi =index_res[i:i+8]
      final_output=final_output+chr(int(bi,2))
      
  return final_output

In [ ]:
result=BinaryToChar()

The ice
The ice cream
The ice cream trucks
It's not
It's not often
Traveling became
[265, 666, 256]
98
97
103


In [ ]:
result

'bag'

In [ ]:
len(result)/len(res)

0.004201680672268907

In [ ]:
phrases=pd.read_csv("Phrases.csv",encoding= 'unicode_escape')
phrases=phrases[0:len(dic)]
phrase_arr=np.array(phrases)
phrase_arr=phrase_arr.reshape((len(dic)))
map_str_num = dict(zip(phrase_arr, dic))

In [ ]:
dupli= [0,0,0,0,0,0,0,0,0,0]
id=[]
for i in range(phrase_arr.shape[0]):
  cnt=0
  zx=str(phrase_arr[i])
  wrds = zx.split(' ')
  cur_wrd=''
  for wrrd in wrds:
    cur_wrd = cur_wrd+wrrd
    if cur_wrd in map_str_num.keys():
      cnt=cnt+1
  
    if wrrd!= wrds[len(wrds)-1]:
      cur_wrd=cur_wrd+' '
    if cur_wrd in map_str_num.keys():
      cnt=cnt+1 
  dupli[cnt]=  dupli[cnt]+1
  if cnt>3:
    id.append(i)



In [ ]:
dupli

[0, 0, 569, 101, 0, 0, 0, 0, 0, 0]

In [ ]:
for i in id:
  print(phrase_arr[i])

In [ ]:
len(dic)

670

In [ ]:
map_int

[[218],
 [432, 60, 231, 284],
 [100, 738, 132],
 [167, 241, 405, 723],
 [767],
 [674],
 [534, 713],
 [135],
 [530, 712],
 [637],
 [139, 601, 505],
 [246, 244, 387, 352],
 [661, 477, 577],
 [321, 602],
 [528, 291],
 [91, 296, 524, 685],
 [572],
 [344, 186],
 [295, 217],
 [673, 424, 228, 192],
 [664],
 [416],
 [618, 756, 657],
 [630, 177, 562],
 [283],
 [491],
 [425, 298],
 [93],
 [163, 138, 555, 57],
 [716, 50, 369],
 [270, 765, 489],
 [761, 391, 645, 672],
 [318, 513],
 [653, 409],
 [584, 389, 278],
 [735, 644, 119, 494],
 [168],
 [210],
 [511, 650, 114, 14],
 [121, 213, 17, 340],
 [223],
 [358, 549, 617],
 [215],
 [452, 760, 550],
 [440],
 [468, 365],
 [676, 582, 433, 235],
 [469, 331, 259],
 [348, 154, 103, 640],
 [336, 5, 374],
 [193, 312, 239, 102],
 [576, 565, 146, 507],
 [458, 422],
 [480, 31, 203, 566],
 [573, 7, 279],
 [351, 401, 15, 517],
 [744, 108, 604],
 [622, 643, 624, 677],
 [200, 219],
 [638, 705],
 [593, 436, 6, 411],
 [680],
 [466, 445],
 [663, 199],
 [629, 62, 174, 75

In [ ]:
map_str_num

{'25 years later': 477,
 '8 percent of 25 is the': 684,
 'A good example': 649,
 'A kangaroo is an': 510,
 'A kangaroo is really': 381,
 'A penny saved': 309,
 'A quiet house': 647,
 'A suit of armor': 683,
 'Abstraction is often': 426,
 'Abstraction is often She tilted her head': 112,
 'After fighting off the': 516,
 'All she wanted was': 598,
 'All they could': 417,
 "Although it wasn't": 135,
 'Aluminum helps to': 576,
 "Art doesn't have": 663,
 'As he entered the church': 369,
 'As she walked': 404,
 'As she walked along': 342,
 'As you consider all': 377,
 'As you consider all the': 653,
 'At last': 119,
 'At that moment': 321,
 'Australian kangaroo': 747,
 'Be careful with': 531,
 'Before he moved': 178,
 'Being unacquainted with': 51,
 'Best friends are': 243,
 'Bill ran from': 274,
 'Blue sounded': 700,
 'Blue sounded too': 442,
 'Boulders lined the': 518,
 'Car safety systems': 0,
 'Carol drank the': 607,
 'Cats are good': 356,
 'Cats are good pets': 537,
 'Check back tomorrow